# Conversational AI with Rasa
![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)

This notebook is based on the [RestaurantBot demo](https://github.com/RasaHQ/rasa_core/blob/master/examples/restaurantbot/)

You'll build a relatively complex bot, that lets you find a restaurant and simulate to book it.

The tutorial consists of three parts:


*   Part 0: Installation and preparations
*   Part 1: Needed files to create your own bot
*   Part 1: You'll start with a basic bot that can only understand natural language but no dialogues.
*   Part 2: You'll add the abilitiy to understand multiturn dialogues.
*   Part 3: Further resources so you can extend this simple demo.


## Part 0: Installation

### Let's start with jupyter configuration

In [67]:
!pip install matplotlib
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Installation of Rasa
First you'll have to install Rasa in this notebook if you already have it installed in your env, you can just skip this. As dependencies, it will install Tensorflow and sklearn-crfsuite among others.

In [68]:
!pip install rasa

### Part 1 Needed files
The most important files are marked with a *.

<style>
td {
  font-size: 50px
}
</style>

| Filename  | Description |
| --- | --- |
| actions.py  | code for your custom actions  |
| config.yml*  | configuration of your NLU and Core models  |
| credentials.yml | details for connecting to other services
| data/nlu.md* | your NLU training data
| data/stories.md* | your stories |
| domain.yml* | your assistant’s domain |
| endpoints.yml	| details for connecting to channels like fb messenger |


### Language Understanding


Lets create some training data here, grouping user messages by their `intent`s. The intent describes what the messages *mean*. [More information about the data format](http://rasa.com/docs/rasa/nlu/training-data-format/). This is the training data for our NLU model, one example per line. Entities are labeled using the markdown link syntex: `[entity value](entity_type)`:

In [78]:
!mkdir data/
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there
- hi, my name is [Carlos](name)
- good morning, i am [Martin](name)
- good evening, i am [Patrick](name)
- hey there, my name is [Peter](name)

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
"""

%store nlu_md > data/nlu.md

mkdir: cannot create directory ‘data/’: File exists
Writing 'nlu_md' (str) to file 'data/nlu.md'.


To properly train your NLU model, you also need to define what is part of that model. Rasa NLU uses a similar pipeline concept as sklearn does. All the components that are listed in the pipeline will be trained one after another and everyone of them contributes its part to the structured data extraction:

In [71]:
config ="""
# Configuration for Rasa NLU.
# https://rasa.com/docs/rasa/nlu/components/
language: en
pipeline: supervised_embeddings

# Configuration for Rasa Core.
# https://rasa.com/docs/rasa/core/policies/
policies:
  - name: MemoizationPolicy
  - name: KerasPolicy
    epochs: 200
  - name: MappingPolicy
"""

%store config > config.yml


Writing 'config' (str) to file 'config.yml'.


In [79]:
!rasa train nlu

Training NLU model...
2019-06-11 17:50:38 INFO     rasa.nlu.training_data.loading  - Training data format of /tmp/tmp8zam3xgm/03138069c3d849fb817751e307da9abf_nlu.md is md
2019-06-11 17:50:38 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 42 (6 distinct intents)
	- Found intents: 'affirm', 'greet', 'goodbye', 'mood_great', 'mood_unhappy', 'deny'
	- entity examples: 4 (1 distinct entities)
	- found entities: 'name'

2019-06-11 17:50:38 INFO     rasa.nlu.model  - Starting to train component WhitespaceTokenizer
2019-06-11 17:50:38 INFO     rasa.nlu.model  - Finished training component.
2019-06-11 17:50:38 INFO     rasa.nlu.model  - Starting to train component RegexFeaturizer
2019-06-11 17:50:38 INFO     rasa.nlu.model  - Finished training component.
2019-06-11 17:50:38 INFO     rasa.nlu.model  - Starting to train component CRFEntityExtractor
2019-06-11 17:50:38 INFO     rasa.nlu.model  - Finished training component.
2019-06-11 17:50:38 INFO     

In [75]:
#rasa run --enable-api -m models/nlu-20190610-191643.tar.gz
!curl localhost:5005/model/parse -d '{"text":"hello"}'

{"intent":{"name":"greet","confidence":0.9686891437},"entities":[],"intent_ranking":[{"name":"greet","confidence":0.9686891437},{"name":"mood_unhappy","confidence":0.0},{"name":"affirm","confidence":0.0},{"name":"deny","confidence":0.0},{"name":"mood_great","confidence":0.0},{"name":"goodbye","confidence":0.0}],"text":"hello"}

In [81]:
#!mkdir models/nlu
#!tar -zxf models/nlu-20190610-191643.tar.gz --directory models/nlu
from rasa.nlu.model import Interpreter

interpreter = Interpreter.load('models/nlu/nlu')
# Parse message to get result.
pprint(interpreter.parse("hello, I am Carlos"))

INFO:tensorflow:Restoring parameters from models/nlu/nlu/component_5_EmbeddingIntentClassifier.ckpt


{
  "intent": {
    "name": "greet",
    "confidence": 0.9113560914993286
  },
  "entities": [
    {
      "start": 12,
      "end": 18,
      "value": "Carlos",
      "entity": "name",
      "confidence": 0.8742144864682851,
      "extractor": "CRFEntityExtractor"
    }
  ],
  "intent_ranking": [
    {
      "name": "greet",
      "confidence": 0.9113560914993286
    },
    {
      "name": "mood_great",
      "confidence": 0.34105661511421204
    },
    {
      "name": "deny",
      "confidence": 0.10866937786340714
    },
    {
      "name": "affirm",
      "confidence": 0.0
    },
    {
      "name": "mood_unhappy",
      "confidence": 0.0
    },
    {
      "name": "goodbye",
      "confidence": 0.0
    }
  ],
  "text": "hello, I am Carlos"
}


In [82]:
!rasa test nlu

2019-06-11 17:54:27 INFO     rasa.nlu.training_data.loading  - Training data format of /tmp/tmp1yyiz8kt/8e8205749d564876a8f860b2879e7b35_nlu.md is md
2019-06-11 17:54:27 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 42 (6 distinct intents)
	- Found intents: 'mood_great', 'affirm', 'deny', 'greet', 'mood_unhappy', 'goodbye'
	- entity examples: 4 (1 distinct entities)
	- found entities: 'name'

2019-06-11 17:54:27 INFO     rasa.nlu.test  - Running model for predictions:
100%|██████████████████████████████████████████| 42/42 [00:00<00:00, 462.06it/s]
2019-06-11 17:54:27 INFO     rasa.nlu.test  - Intent evaluation results:
2019-06-11 17:54:27 INFO     rasa.nlu.test  - Intent Evaluation: Only considering those 42 examples that have a defined intent out of 42 examples
2019-06-11 17:54:27 INFO     rasa.nlu.test  - F1-Score:  1.0
2019-06-11 17:54:27 INFO     rasa.nlu.test  - Precision: 1.0
2019-06-11 17:54:27 INFO     rasa.nlu.test  - Accuracy:  1.0

# Part 2: Adding dialogue capabilities
### Writing Stories

A good place to start is by writing a few stories. These are example conversations that Rasa Core will learn from. 

The format works like this:

A story starts with `##` and you can give it a name. 
lines that start with `*` are messages sent by the user. Although you don't write the *actual* message, but rather the intent (and the entities) that represent what the user *means*. If you don't know about intents and entities, don't worry! We will talk about them more later. 
Lines that start with `-` are *actions* taken by your bot. In this case all of our actions are just messages sent back to the user, like `utter_greet`, but in general an action can do anything, including calling an API and interacting with the outside world. 

In [97]:
stories_md = """

## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
##just hi
* greet{"name" : "Carlos"}
  - utter_greet_with_name

## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - action_retrieve_image
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - action_retrieve_image
  - utter_cheer_up
  - utter_did_that_help
* mood_deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye


## Generated Story 7314638820442985887
* greet
    - utter_greet
* mood_unhappy
    - action_retrieve_image
    - slot{"img_api_response": "https://picsum.photos/200/300/?random"}
    - utter_cheer_up
    - utter_did_that_help
* deny
    - utter_goodbye
    - action_restart
    
## Generated Story 7557684988670147894
* greet{"name": "Carlos"}
    - slot{"name": "Carlos"}
    - utter_greet_with_name
* mood_great
    - utter_happy
* goodbye
    - utter_goodbye
    - action_restart

"""

%store stories_md > data/stories.md

Writing 'stories_md' (str) to file 'data/stories.md'.


### Defining a Domain

The domain specifies the universe that your bot lives in. You should list all of the intents and actions that show up in your stories. 
This is also the place to write templates, which contain the messages your bot can send back

In [95]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy

slots:
  img_api_response:
    type: unfeaturized
    
  name:
    type: text

actions:
- utter_greet
- utter_greet_with_name
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye
- action_retrieve_image

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_greet_with_name:
  - text: "Hey! How are you {name}?"

  utter_cheer_up:
  - text: "Here is something to cheer you up: {img_api_response}"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


### Adding Custom API methods
Sometimes, you not only want to send back messages to the user, but you also want to call an API or run some code. YOu can create custom actions that will be called once the bots ML model predicts them. You'll use that to fetch a random image from a remote server (we are not actually fetching it here, but we could ;)

In [87]:
actions= """
from rasa_sdk import Action, Tracker
from rasa_sdk.events import SlotSet

class ActionRetrieveImage(Action):
   def name(self):
      return "action_retrieve_image"

   def run(self, dispatcher, tracker, domain):
        dispatcher.utter_message("looking for a good img")
        url = "https://picsum.photos/200/300/?random"
        return [SlotSet("img_api_response", url)]  
 
"""

%store actions > actions.py

endpoints = """
action_endpoint:
  url: http://localhost:5055/webhook
"""
%store endpoints > endpoints.yml

Writing 'actions' (str) to file 'actions.py'.
Writing 'endpoints' (str) to file 'endpoints.yml'.


## Training your Dialogue Model
Now comes the fun part! We're going to show Rasa Core the stories we wrote above, and train a model on these examples. In this case, the model is a neural network implemented in Keras which learns to predict which action to take next.

In [96]:
!rasa train

2019-06-11 18:11:27 INFO     rasa.model  - Data (domain) for Core model changed.
2019-06-11 18:11:27 INFO     rasa.model  - Data (messages) for NLU model changed.
Training Core model...
2019-06-11 18:11:27 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2019-06-11 18:11:27 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
2019-06-11 18:11:28 WARNING  rasa.core.training.dsl  - Found unknown intent 'deny' on line 50. Please, make sure that all intents are listed in your domain yaml.
Processed trackers: 100%|█████████| 6/6 [00:00<00:00, 1794.99it/s, # actions=24]
Processed actions: 24it [00:00, 8727.53it/s, # examples=24]
Processed trackers: 100%|█████| 297/297 [00:00<00:00, 805.66it/s, # actions=225]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 22)             0         
___________

225/225 [==============================] - 0s 345us/sample - loss: 0.5789 - acc: 0.8133
Epoch 61/200
225/225 [==============================] - 0s 270us/sample - loss: 0.5399 - acc: 0.8533
Epoch 62/200
225/225 [==============================] - 0s 318us/sample - loss: 0.5436 - acc: 0.8311
Epoch 63/200
225/225 [==============================] - 0s 240us/sample - loss: 0.5221 - acc: 0.8489
Epoch 64/200
225/225 [==============================] - 0s 217us/sample - loss: 0.4979 - acc: 0.8578
Epoch 65/200
225/225 [==============================] - 0s 229us/sample - loss: 0.5225 - acc: 0.8222
Epoch 66/200
225/225 [==============================] - 0s 227us/sample - loss: 0.4784 - acc: 0.8400
Epoch 67/200
225/225 [==============================] - 0s 223us/sample - loss: 0.4695 - acc: 0.8356
Epoch 68/200
225/225 [==============================] - 0s 385us/sample - loss: 0.4696 - acc: 0.8711
Epoch 69/200
225/225 [==============================] - 0s 487us/sample - loss: 0.4705 - acc: 0.8400
Epo

225/225 [==============================] - 0s 344us/sample - loss: 0.2081 - acc: 0.9333
Epoch 141/200
225/225 [==============================] - 0s 403us/sample - loss: 0.2321 - acc: 0.8844
Epoch 142/200
225/225 [==============================] - 0s 259us/sample - loss: 0.2199 - acc: 0.9111
Epoch 143/200
225/225 [==============================] - 0s 239us/sample - loss: 0.2318 - acc: 0.8933
Epoch 144/200
225/225 [==============================] - 0s 233us/sample - loss: 0.2211 - acc: 0.9244
Epoch 145/200
225/225 [==============================] - 0s 215us/sample - loss: 0.2187 - acc: 0.9067
Epoch 146/200
225/225 [==============================] - 0s 246us/sample - loss: 0.2142 - acc: 0.9289
Epoch 147/200
225/225 [==============================] - 0s 246us/sample - loss: 0.1977 - acc: 0.9200
Epoch 148/200
225/225 [==============================] - 0s 260us/sample - loss: 0.2087 - acc: 0.9200
Epoch 149/200
225/225 [==============================] - 0s 231us/sample - loss: 0.2429 - acc: 0

Epochs: 100%|█████████| 300/300 [00:02<00:00, 148.87it/s, loss=0.091, acc=1.000]
2019-06-11 18:11:51 INFO     rasa.nlu.classifiers.embedding_intent_classifier  - Finished training embedding classifier, loss=0.091, train accuracy=1.000
2019-06-11 18:11:51 INFO     rasa.nlu.model  - Finished training component.
2019-06-11 18:11:51 INFO     rasa.nlu.model  - Successfully saved model into '/tmp/tmpq62myz8z/nlu'
NLU model training completed.
Your Rasa model is trained and saved at '/home/csegura/repo/chatbot/RasaDemo/models/20190611-181151.tar.gz'.


### Pro Tip: Visualising the Training Data

You can visualise the stories to get a sense of how the conversations go. This is usually a good way to see if there are any stories which don't make sense


In [90]:
#you may need any of these to be able to display the conversations graph
#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!brew install graphviz;
!rasa visualize

2019-06-11 18:01:59 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2019-06-11 18:01:59 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
2019-06-11 18:02:00 INFO     rasa.nlu.training_data.loading  - Training data format of /tmp/tmptk_wjys2/c0da753598cb4cb3a7d62a6bd09de0b7_nlu.md is md
2019-06-11 18:02:00 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 42 (6 distinct intents)
	- Found intents: 'goodbye', 'greet', 'deny', 'mood_great', 'mood_unhappy', 'affirm'
	- entity examples: 4 (1 distinct entities)
	- found entities: 'name'

2019-06-11 18:02:00 INFO     rasa.core.visualize  - Starting to visualize stories...
Processed Story Blocks: 100%|█████| 4/4 [00:00<00:00, 2293.54it/s, # trackers=1]
2019-06-11 18:02:01 INFO     rasa.core.visualize  - Finished graph creation. Saved into file:///home/csegura/repo/chatbot/RasaDemo/graph.html


## Testing the bot


First run the actions endpoint server and then run "rasa shell" in the terminal.


In [92]:
%%script bash --bg 
rasa run actions


In [65]:
#this must run in the reminal
#!rasa shell

In [91]:
#When its finished, we can stop all background scripts with
%killbgscripts

All background processes were killed.


## Adding new stories

This must run in the terminal


rasa interactive --endpoints endpoints.yml
